In [ ]:
# BOtCH: улучшенный чат-бот с генеративной моделью DialoGPT
В этом ноутбуке реализован бот с DialoGPT, базовая оценка качества и демонстрация.


In [ ]:
!pip install transformers torch nltk matplotlib --quiet


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
import matplotlib.pyplot as plt
import random

nltk.download('punkt')


In [ ]:
model_name = 'microsoft/DialoGPT-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:
def generate_response(history, user_input, max_length=1000):
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([history, new_user_input_ids], dim=-1) if history is not None else new_user_input_ids
    output_ids = model.generate(bot_input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    response_ids = output_ids[:, bot_input_ids.shape[-1]:]
    response_text = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return output_ids, response_text


In [ ]:
history = None
user_inputs = [
    "Привет!",
    "Как тебя зовут?",
    "Расскажи шутку",
    "Спасибо, пока!"
]

for user_input in user_inputs:
    history, bot_response = generate_response(history, user_input)
    print(f"Пользователь: {user_input}")
    print(f"БОТ: {bot_response}")
    print('---')


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

test_data = [
    ("Привет!", "Привет! Как я могу помочь?"),
    ("Как тебя зовут?", "Я бот, меня зовут BOtCH."),
    ("Расскажи шутку", "Почему программисты любят природу? Потому что там много багов!"),
    ("Пока", "До свидания!")
]

bleu_scores = []
for question, reference in test_data:
    _, response = generate_response(None, question)
    reference_tokens = nltk.word_tokenize(reference.lower())
    response_tokens = nltk.word_tokenize(response.lower())
    score = sentence_bleu([reference_tokens], response_tokens)
    bleu_scores.append(score)
    print(f"Вопрос: {question}")
    print(f"Ответ бота: {response}")
    print(f"Эталон: {reference}")
    print(f"BLEU: {score:.3f}")
    print('---')


In [ ]:
plt.figure(figsize=(8,4))
plt.bar(range(len(bleu_scores)), bleu_scores, tick_label=[q for q, _ in test_data])
plt.title('BLEU scores for test questions')
plt.ylabel('BLEU score')
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.show()
